### Guide to the notebook

__The notebook in a nutshell:__ Imported the aggregated DF and applied Random Forest thrice using different Hyperparameters and noted the RMSE on the validation as well as the test set. Next, Linear Regression was applied, and lastly, a small Neural Net was applied. Since the validation and test scores formed around about a straight line (their variance was similar), the chosen validation set was exported for future use.

### Importing Libs

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from sklearn import svm

Using TensorFlow backend.


### Importing & Fixing Datasets

In [2]:
Xtrain = pd.read_pickle('XtrainRF'); Ytrain = pd.read_pickle('YtrainRF'); Xtest = pd.read_pickle('XtestRF')

Xtrain[['feature_1', 'feature_2']] = Xtrain[['feature_1', 'feature_2']].astype('category')

Xtrain[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']]\
=Xtrain[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']].astype('int32')

Xtest[['feature_1', 'feature_2']] = Xtest[['feature_1', 'feature_2']].astype('category')

Xtest[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']]\
=Xtest[['Cat_1_Y','Cat2Tot1', 'Cat2Tot2', 'Cat2Tot3', 'Cat2Tot4', 'Cat3TotA', 'Cat3TotB']].astype('int32')

### Creating Vars

In [11]:
# Make Vars
X=Train.loc[:,Train.columns!='target']
Y=Train.loc[:,'target']

Xtrain,Xval,Ytrain,Yval=train_test_split(Xtrain,Ytrain,test_size=0.20)

Xtrain.to_pickle('Xtrain');Xval.to_pickle('Xval');Ytrain.to_pickle('Ytrain');Yval.to_pickle('Yval')

Xtrain.first_active_month=Xtrain.first_active_month.astype('category');Xtrain.first_active_month=Xtrain.first_active_month.cat.codes;Xtrain

Xval.first_active_month=Xval.first_active_month.astype('category');Xval.first_active_month=Xval.first_active_month.cat.codes;Xval

XtrainRf1=Xtrain.loc[:,Xtrain.columns!='card_id'];XvalRf1=Xval.loc[:,Xval.columns!='card_id']

Xtest=Test.loc[:,Test.columns!='card_id']

Xtest.first_active_month=Xtest.first_active_month.astype('category').cat.codes;Xtest.describe(include='all')

### Applying 1st RF

In [16]:
RF=RandomForestRegressor(n_estimators=100,max_features=0.5,min_samples_leaf=1,bootstrap=False,n_jobs=-1)

modelRF=RF.fit(Xtrain,Ytrain)

RF1.oob_score_

Ypred=RF.predict(Xval)

#### RMSE of val set

In [7]:
# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.714320993586702

In [13]:
# RMSE 2nd Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.704414994858926

In [19]:
# RMSE 3rd Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.702026929565763

#### RMSE of test set

In [20]:
Ypred=RF.predict(Xtest)

In [21]:
Ypred=pd.Series(Ypred);Ypred.to_csv('Ypred.csv')

C:\Users\AMS\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


__RMSE Test Set 1st Check: 3.97082__

__RMSE Test Set 2nd Check: 3.96522__

__RMSE Test Set 3rd Check: 3.96388__

### Applying LinReg

#### One Hot Encoding

In [4]:
XtrainLR=pd.get_dummies(Xtrain,columns=['feature_1','feature_2','feature_3'],drop_first=True);XtrainLR

,purchase_amount,Cat_1_Y,Cat2Tot1,Cat2Tot2,Cat2Tot3,Cat2Tot4,Cat3TotA,Cat3TotB,AvgPurAmt,TimeSpent,...,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_2,feature_2_3,feature_3_1
7607,-90.436539,0,133,0,0,0,138,0,-0.655337,321,...,186.619254,918.0,189.250000,1,0,0,0,1,0,0
98805,-7.680020,1,1,0,23,0,0,18,-0.307201,137,...,113.798870,276.0,82.625000,0,0,0,1,0,0,1
112580,6.454022,1,16,0,0,0,0,3,0.379648,788,...,24.322352,144.0,28.421875,0,0,0,1,0,0,1
75235,-31.904755,20,6,0,10,16,0,40,-0.601977,1091,...,101.661288,354.0,76.937500,0,1,0,0,0,1,1
9519,-23.167635,0,54,0,0,0,54,0,-0.429030,1176,...,566.143028,336.0,229.625000,0,1,0,0,0,1,1
159342,-198.776230,0,303,0,0,0,303,0,-0.656027,782,...,976.063271,1872.0,702.500000,1,0,0,0,1,0,0
133812,-47.988564,0,83,0,0,0,86,0,-0.558007,621,...,282.550212,606.0,237.875000,1,0,0,0,1,0,0
59907,-20.613729,0,3,0,27,0,31,0,-0.664959,139,...,50.881140,192.0,55.000000,0,1,0,0,1,0,1
126312,-13.332753,0,1,0,0,0,20,0,-0.666638,206,...,30.735406,156.0,32.937500,0,1,0,0,1,0,1
29250,-42.230087,0,66,0,0,0,0,65,-0.621031,934,...,180.250895,504.0,135.000000,0,0,0,1,0,0,1


In [5]:
XvalLR=pd.get_dummies(Xval, columns=['feature_1','feature_2','feature_3'],drop_first=True);XvalLR

,purchase_amount,Cat_1_Y,Cat2Tot1,Cat2Tot2,Cat2Tot3,Cat2Tot4,Cat3TotA,Cat3TotB,AvgPurAmt,TimeSpent,...,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_2,feature_2_3,feature_3_1
165253,4.273555,0,10,0,0,0,0,1,0.427355,348,...,10.490757,60.0,10.437500,1,0,0,0,0,0,0
179289,-65.600777,0,1,95,0,1,0,86,-0.643145,481,...,118.667985,612.0,98.187500,0,1,0,0,0,0,1
10136,-73.182251,0,14,0,95,0,110,0,-0.665293,394,...,232.249434,750.0,187.000000,0,1,0,0,1,0,1
108009,-10.215924,0,14,0,0,0,14,0,-0.729709,595,...,19.504611,114.0,19.156250,0,1,0,0,1,0,1
82678,-425.168884,0,704,0,0,0,704,0,-0.603933,625,...,1910.261851,4400.0,1285.000000,0,0,0,1,1,0,1
174336,-35.016670,0,51,0,0,0,0,46,-0.686601,194,...,71.821368,342.0,65.562500,1,0,0,0,0,0,0
40640,-60.795944,0,14,0,87,0,110,0,-0.552690,654,...,152.228325,786.0,148.125000,1,0,0,0,1,0,0
94024,-14.409358,5,25,0,0,0,0,29,-0.480312,239,...,33.556834,192.0,32.437500,0,0,1,0,0,0,0
88755,-16.040094,0,25,0,0,0,0,22,-0.641604,280,...,43.939082,162.0,39.156250,0,0,1,0,0,1,0
200945,-71.036438,0,19,0,0,0,100,0,-0.710364,149,...,151.018778,612.0,133.125000,0,1,0,0,0,0,1


In [6]:
XtestLR=pd.get_dummies(Xtest,columns=['feature_1','feature_2','feature_3'],drop_first=True);XtestLR

,purchase_amount,Cat_1_Y,Cat2Tot1,Cat2Tot2,Cat2Tot3,Cat2Tot4,Cat3TotA,Cat3TotB,AvgPurAmt,TimeSpent,...,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_2,feature_2_3,feature_3_1
0,-29.815680,10,37,0,0,0,0,37,-0.634376,333.0,...,123.725782,444.0,109.062500,0,1,0,0,0,1,1
1,-54.349330,3,0,0,1,82,0,83,-0.631969,474.0,...,106.378598,528.0,97.250000,1,0,0,0,0,1,0
2,-0.714405,2,1,0,0,0,0,2,-0.064946,224.0,...,16.508767,90.0,14.820312,0,0,0,1,0,0,1
3,-19.434389,1,34,0,0,0,0,22,-0.539844,137.0,...,36.856241,216.0,36.968750,1,0,0,0,0,0,0
4,52.766408,0,4,0,5,84,0,86,0.567381,863.0,...,307.189420,714.0,346.000000,0,0,0,1,0,0,1
5,-1.357119,1,7,0,5,0,0,11,-0.104394,254.0,...,49.261161,114.0,32.562500,0,0,0,1,0,0,1
6,-22.922298,1,1,9,0,0,0,34,-0.402146,205.0,...,58.666847,334.0,59.718750,0,0,0,0,1,0,0
7,-10.453508,0,15,0,0,0,15,0,-0.696901,356.0,...,38.082704,120.0,31.593750,0,1,0,0,0,0,1
8,-13.659610,9,4,0,24,0,0,19,-0.359463,301.0,...,104.611489,354.0,77.187500,0,0,1,0,0,1,0
9,-32.378211,0,52,0,0,0,0,51,-0.622658,628.0,...,117.225847,348.0,109.875000,0,0,0,1,0,0,1


#### Fitting model

In [25]:
LR=LinearRegression(n_jobs=-1)

In [26]:
model=LR.fit(XtrainLR,Ytrain)

In [27]:
Ypred=LR.predict(XvalLR)

#### RMSE of val set

In [28]:
# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

1.6704172723870974

#### RMSE of test set

In [29]:
Ypred=LR.predict(XtestLR)

In [30]:
Ypred=pd.Series(Ypred);Ypred.to_csv('Ypred.csv')

C:\Users\AMS\Anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


__RMSE Test Set 1st Check: 3.94779__

### Applying NN

In [42]:
# introduces randomness. 
# fix random seed for reproducibility
np.random.seed(7)
# load pima indians dataset

# split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(100, input_dim=27, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['mse']) #metrics should be diff becuase of skewed dataset
# Fit the model
model.fit(XtrainLR, Ytrain, epochs=100, batch_size=10000)
# evaluate the model
scores = model.evaluate(XtrainLR, Ytrain)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Ypred=model.predict(XvalLR)

# RMSE 1st Check
MSE=mean_squared_error(Yval,Ypred)
RMSE=sqrt(MSE);RMSE

Epoch 1/100
159684/159684 [==============================] - 1s 8us/step - loss: 30316.2305 - mean_squared_error: 30316.2305
Epoch 2/100
159684/159684 [==============================] - 1s 4us/step - loss: 684.7337 - mean_squared_error: 684.7337
Epoch 3/100
159684/159684 [==============================] - 1s 4us/step - loss: 176.3462 - mean_squared_error: 176.3462
Epoch 4/100
159684/159684 [==============================] - 1s 4us/step - loss: 77.8561 - mean_squared_error: 77.8561
Epoch 5/100
159684/159684 [==============================] - 1s 3us/step - loss: 49.4828 - mean_squared_error: 49.4828
Epoch 6/100
159684/159684 [==============================] - 1s 3us/step - loss: 36.3288 - mean_squared_error: 36.3288
Epoch 7/100
159684/159684 [==============================] - 1s 3us/step - loss: 31.7011 - mean_squared_error: 31.7011
Epoch 8/100
159684/159684 [==============================] - 1s 3us/step - loss: 27.5002 - mean_squared_error: 27.5002
Epoch 9/100
159684/159684 [===========

#### RMSE of test set

In [45]:
Ypred=model.predict(XtestLR)

Ypred=pd.DataFrame(Ypred);Ypred.to_csv('Ypred.csv')

__RMSE Test Set 1st Check: 3.95983__

__RMSE Test Set 2nd Check: 16.26424__

__RMSE Test Set 3rd Check: 5.92523__

__RMSE Test Set 4th Check: 5.61297__

### Exporting Data

In [7]:
Xtrain.to_pickle('XtrainRFM16');Xval.to_pickle('XvalRFM16');Xtest.to_pickle('XtestRFM16')

XtrainLR.to_pickle('XtrainLRM16');XvalLR.to_pickle('XvalLRM16');XtestLR.to_pickle('XtestLRM16')

Ytrain.to_pickle('YtrainRFM16');Yval.to_pickle('YvalRFM16')